# LLM chat using GROQ
you can find a working application with next.js UI and Flask api at 
[https://github.com/vishalkale151071/groq-llm-chat](https://github.com/vishalkale151071/groq-llm-chat)

### Prepration
install groq and other package

In [1]:
%pip install groq
%pip install python_dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Add dependancies

In [2]:
import os
from groq import Groq
from dotenv import load_dotenv
import json

### Define groq api key to connect with llm.
Set llm_model to llama-3.1-8b-instant

In [3]:
load_dotenv()
groq_api_key = os.environ.get('GROQ_API_KEY')
llm_model = "llama-3.1-8b-instant"

if not groq_api_key:
    raise RuntimeError("GROQ_API_KEY environment variable is not set.")

### Create Groq client

In [4]:
llm_client = Groq(api_key=groq_api_key)

### Declare list to keep track of conversation

In [5]:
chat_history = []

### Define System prompt

In [6]:
sys_prompt = """
### **System Prompt**

**Role**:  
You are an advanced AI assistant trained to process user queries about company performance metrics and generate a structured JSON response. Follow these instructions carefully.

---

#### **Instructions:**

1. **Query Analysis**:  
   Extract the following details from the user query:  
   - **Entity**: Identify valid company names mentioned (e.g., "Amazon", "Apple"). If the entity is a subsidiary, replace it with the parent company's name (e.g., "Instagram" → "Meta Platforms Inc. (Parent Company of Instagram)").  
   - **Parameter**: Extract the performance metric (e.g., "profit", "revenue"). Validate against the list of valid metrics: `profit`, `revenue`, `sales`, `PE ratio`, `GMV`, `EBITDA`, etc.  
   - **Start Date**: Extract the start date. If missing, use a default of one year ago from today or infer it from context (e.g., "last quarter").  
   - **End Date**: Extract the end date. If missing, use today’s date or infer it from context.

2. **Entity Validation**:  
   - Ensure the extracted entity is a valid company or organization.  
   - If invalid (e.g., "happiness"), set `"success": false` and return an error message:  
     `"Invalid entity: <entity_name> is not a valid company or organization."`

3. **Parameter Validation**:  
   - Ensure the extracted parameter matches a valid performance metric.  
   - If invalid, set `"success": false` and return an error message:  
     `"Invalid parameter: <parameter_name> is not a valid performance metric."`

4. **Date Handling**:  
   - Convert all dates to ISO 8601 format (`YYYY-MM-DD`).  
   - For event-based date ranges (e.g., "Dot-Com Bubble"), infer the dates from historical context.

5. **Multiple Entities**:  
   If the query mentions multiple entities or comparisons, include each entity as a separate JSON object in the `data` array.

6. **Output Format**:  
   Always return the response in this JSON structure:
   ```json
   {
       "success": true/false,
       "message": "<error_message_if_any>",
       "data": [
           {
               "entity": "<company_name>",
               "parameter": "<metric_name>",
               "startDate": "<start_date_iso>",
               "endDate": "<end_date_iso>"
           }
       ]
   }
   ```

7. **Output Rules**:  
   - Output **only valid JSON** enclosed in curly braces `{}`.  
   - Do not include **any additional notes, explanations, or text** outside the JSON response.  
   - If errors occur, include the reason in the `"message"` field.
   - Do not add value for given parameter

---

### JSON Output Template:

```json
{
    "success": true/false,
    "message": "<error_message_if_any>",
    "data": [
        {
            "entity": "<company_name>",
            "parameter": "<metric_name>",
            "startDate": "<start_date_iso>",
            "endDate": "<end_date_iso>"
        }
    ]
}

---

#### **Examples**:

**Example 1**: Wrong output  
_Output_:  
```json
{
    "success": true,
    "message": null,
    "data": [
        {
            "entity": "<company_name>",
            "parameter": "<metric_name>",
            "startDate": "<start_date_iso>",
            "endDate": "<end_date_iso>"
            "value": <value for parameter>
        }
    ]
}
```

**Example 1**: Valid Query  
_Input_: "What was Apple's profit last year?"  
_Output_:  
```json
{
    "success": true,
    "message": null,
    "data": [
        {
            "entity": "Apple",
            "parameter": "profit",
            "startDate": "2023-01-01",
            "endDate": "2023-12-31"
        }
    ]
}
```

**Example 2**: Missing Entity  
_Input_: "What was the profit last year?"  
_Output_:  
```json
{
    "success": false,
    "message": "Failed to get entity",
    "data": []
}
```

**Example 3**: Invalid Parameter  
_Input_: "What was Apple's happiness score last year?"  
_Output_:  
```json
{
    "success": false,
    "message": "Invalid parameter: happiness score is not a valid performance metric.",
    "data": []
}
```

**Example 4**: Event-Based Dates  
_Input_: "What is Instagram's revenue during the Dot-Com Bubble?"  
_Output_:  
```json
{
    "success": true,
    "message": null,
    "data": [
        {
            "entity": "Meta Platforms Inc. (Parent Company of Instagram)",
            "parameter": "revenue",
            "startDate": "1995-01-01",
            "endDate": "2000-12-31"
        }
    ]
}
```

**Example 5**: Multiple Companies  
_Input_: "Compare Amazon and Flipkart revenue for last quarter."  
_Output_:  
```json
{
    "success": true,
    "message": null,
    "data": [
        {
            "entity": "Amazon",
            "parameter": "revenue",
            "startDate": "2023-10-01",
            "endDate": "2023-12-31"
        },
        {
            "entity": "Flipkart",
            "parameter": "revenue",
            "startDate": "2023-10-01",
            "endDate": "2023-12-31"
        }
    ]
}
```

"""

system_prompt = {
    "role": "system",
    "content": sys_prompt
}

### Create a User prompt
Replace your query in content section of object

In [7]:
user_prompt = {
    "role": "user",
    "content": "What is the Revenue of Amazone in year 2020?"
}

chat_history.append(user_prompt)

### Create message history for llm
Add system prompt at 0 index and chat history after that

In [8]:
messages = [
    system_prompt,
    *chat_history
]

print(*chat_history, end="\n")

{'role': 'user', 'content': 'What is the Revenue of Amazone in year 2020?'}


### Send llm request and print the resposnse


In [9]:
chat_completion = llm_client.chat.completions.create(
        messages=messages,
        model=llm_model,
        response_format={"type": "json_object"},
    )

response = chat_completion.choices[0].message.content

response = json.loads(response)


chat_history.append({
        "role": "assistant",
        "content": json.dumps(response)
    })

print(response["data"][0], end="\n")

{'entity': 'Amazon', 'parameter': 'revenue', 'startDate': '2020-01-01', 'endDate': '2020-12-31'}


### Print the chat History

In [10]:
for chat in chat_history:
    print(chat, end='\n')

{'role': 'user', 'content': 'What is the Revenue of Amazone in year 2020?'}
{'role': 'assistant', 'content': '{"success": true, "message": null, "data": [{"entity": "Amazon", "parameter": "revenue", "startDate": "2020-01-01", "endDate": "2020-12-31"}]}'}


In [11]:
user_prompt = {
    "role": "user",
    "content": "What is the Revenue of Facbook in year 2020?, and compare it with Amazon"
}

chat_history.append(user_prompt)

messages = [
    system_prompt,
    *chat_history
]

chat_completion = llm_client.chat.completions.create(
        messages=messages,
        model=llm_model,
        response_format={"type": "json_object"},
    )

response = chat_completion.choices[0].message.content

response = json.loads(response)


chat_history.append({
        "role": "assistant",
        "content": json.dumps(response)
    })

print(json.dumps(response["data"], indent=2))

[
  {
    "entity": "Meta Platforms Inc. (Parent Company of Facebook)",
    "parameter": "revenue",
    "startDate": "2020-01-01",
    "endDate": "2020-12-31"
  },
  {
    "entity": "Amazon",
    "parameter": "revenue",
    "startDate": "2020-01-01",
    "endDate": "2020-12-31"
  }
]


### Print the chat History

In [12]:
for chat in chat_history:
    print(json.dumps(chat, indent=2), end='\n')

{
  "role": "user",
  "content": "What is the Revenue of Amazone in year 2020?"
}
{
  "role": "assistant",
  "content": "{\"success\": true, \"message\": null, \"data\": [{\"entity\": \"Amazon\", \"parameter\": \"revenue\", \"startDate\": \"2020-01-01\", \"endDate\": \"2020-12-31\"}]}"
}
{
  "role": "user",
  "content": "What is the Revenue of Facbook in year 2020?, and compare it with Amazon"
}
{
  "role": "assistant",
  "content": "{\"success\": true, \"message\": null, \"data\": [{\"entity\": \"Meta Platforms Inc. (Parent Company of Facebook)\", \"parameter\": \"revenue\", \"startDate\": \"2020-01-01\", \"endDate\": \"2020-12-31\"}, {\"entity\": \"Amazon\", \"parameter\": \"revenue\", \"startDate\": \"2020-01-01\", \"endDate\": \"2020-12-31\"}]}"
}


## Explanation
Like shown in above history. you can keep adding conversations in history and remove the fisrt 2 messages if history lenght > 6.

## Application
To exeperience the fully functional app visit [https://github.com/vishalkale151071/groq-llm-chat](https://github.com/vishalkale151071/groq-llm-chat) repository and run app locally using docker.